## Test hybrid PT-JPL functions 

These functions have been adapted to work both for numpy arrays and ee.Images

In [1]:
import ee
ee.Initialize()
import geemap
%load_ext autoreload
%autoreload 2

## compute_fwet

This function computes relative surface wetness from relative humidity. First we test it with a simple numpy array:

In [45]:
import numpy as np
from geeet.ptjpl import compute_fwet
compute_fwet(np.array([52])) # test with numpy array

array([0.07311616])

We now retrieve some test RH EE data, e.g. band `relative_humidity_2m_above_ground` from the [NOAA/GFS0P25](https://developers.google.com/earth-engine/datasets/catalog/NOAA_GFS0P25#bands). First we display it:

In [3]:
date_str='2017-04-15' 
lonlat=[38.530867, 30.236383] 
date = ee.Date(date_str)
geom = ee.Geometry.Point(lonlat[0], lonlat[1]) # lon/lat

test_img = ee.Image(
  ee.ImageCollection('NOAA/GFS0P25')
    .filterBounds(geom)
    .filterDate(date,date.advance(3,'month'))
    .sort('system:time_start')
    .first()
  )

Map = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=5)
Map.addLayer(test_img, {'bands':['relative_humidity_2m_above_ground'],'min':0, 'max':100, 'palette':["440154","3a528b","20908d","5dc962","fde725"], 'opacity': 0.8}, 'Relative humidity')
Map

Map(center=[30.236383, 38.530867], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

And now we test the function. Note that we need to **select** the appropriate band:

In [4]:
rh_band = 'relative_humidity_2m_above_ground'
fwet_img = compute_fwet(test_img.select(rh_band), 'fwet') # input = ee.Image -> output = ee.Image
# and we add it to the map above for visualization.
Map.addLayer(fwet_img, {'bands':['fwet'],'min':0, 'max':1, 'palette':["440154","3a528b","20908d","5dc962","fde725"], 'opacity': 0.8 }, 'computed fwet')

## compute_fapar

This function computes the fraction of the photosynthetic active radiation (PAR) absorbed by green vegetation cover.

In [5]:
from geeet.ptjpl import compute_fapar
compute_fapar(0.790563821) 

0.77570477625

We now retrieve some test NDVI data from EE, e.g. band `NDVI` from the [MODIS/MCD43A4_006_NDVI](https://developers.google.com/earth-engine/datasets/catalog/MODIS_MCD43A4_006_NDVI) dataset. First we display it:

In [6]:
NDVI = ee.ImageCollection('MODIS/MCD43A4_006_NDVI') \
                  .filter(ee.Filter.date('2018-04-01', '2018-05-01')).first() \
                  .select("NDVI")
import geemap.colormaps as cm
ndvi_palette = cm.palettes.ndvi
Map2 = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=9)
Map2.addLayer(NDVI, {'min':0, 'max':1, 'palette':ndvi_palette, 'opacity': 0.8}, 'NDVI')
Map2

Map(center=[30.236383, 38.530867], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [7]:
fapar = compute_fapar(NDVI, band_name = 'fapar')
pal = cm.palettes.YlOrRd
Map2.addLayer(fapar, {'min':0, 'max':1, 'palette':pal, 'opacity': 0.8}, 'fapar')

## compute_fipar

This function computes the fraction of the photosynthetic active radiation (PAR) intercepted by total vegetation cover. We can use the same NDVI test data.

In [8]:
from geeet.ptjpl import compute_fipar
compute_fipar(0.7905638) 

array(0.7405638)

In [9]:
fipar = compute_fipar(NDVI, band_name = 'fipar')
Map2.addLayer(fipar, {'min':0, 'max':1, 'palette':pal, 'opacity': 0.8}, 'fipar')

## compute_fg

This function computes the green canopy fraction. We can use the same NDVI test data.

In [10]:
from geeet.ptjpl import compute_fg
compute_fg(0.435778)

array(0.86117534)

In [11]:
fg = compute_fg(NDVI, band_name = 'fg')
Map2.addLayer(fg, {'min':0, 'max':1, 'palette':ndvi_palette, 'opacity': 0.8}, 'fg')

## compute_ft_arid
This function computes the plant temperature constraint Ft.

To test this, we need some air temperature data. Let's get `temperature_2m` from the `ECMWF/ERA5_LAND/HOURLY` dataset.

In [12]:
date_time_str='2017-04-15T13:00:00' 
date_time = ee.Date(date_time_str)
print(date_time_str.format('z'))

2017-04-15T13:00:00


In [13]:
climate_img = ee.Image(
  ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
    .filterBounds(geom)
    .filterDate(date,date.advance(1,'hour'))
    .sort('system:time_start')
    .first()
  )
t2m = 'temperature_2m'
tair = climate_img.select(t2m).subtract(273.15)
Map3 = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=9)
Map3.addLayer(tair, {'min':0, 'max':35, 'palette':pal, 'opacity': 0.8}, 'Air temperature (2m)')
Map3

Map(center=[30.236383, 38.530867], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [14]:
from geeet.ptjpl import compute_ft_arid
compute_ft_arid(18.8569)

array(0.79760302)

In [15]:
ft = compute_ft_arid(tair, 'ft')
Map3.addLayer(ft,{'min':0, 'max':35, 'palette':pal, 'opacity': 0.8}, 'ft')

## compute_fm

This function computes the plant moisture constraint. It requires `f_apar` and `f_aparmax`. To easily retrieve `f_aparmax`, the function `add_fapar` can be mapped to an ee.ImageCollection. Here we demonstrate how to compute `f_aparmax` for one year of NDVI data:

In [16]:
NDVI_one_year = ee.ImageCollection('MODIS/MCD43A4_006_NDVI') \
                  .filter(ee.Filter.date('2018-01-01', '2018-12-01')) \
                  .select("NDVI")
NDVI_one_year

In [17]:
from geeet.ptjpl import add_fapar
fapar_max = NDVI_one_year.map(add_fapar).select('fapar').reduce(ee.Reducer.max())
fapar_max

In [18]:
Map2.addLayer(fapar_max, {'min':0, 'max':1, 'palette':pal, 'opacity': 0.8}, 'fapar_max')

Now we can compute_fm:

In [19]:
from geeet.ptjpl import compute_fm
compute_fm(0.736, 0.7924)

array(0.92882383)

In [20]:
fm = compute_fm(fapar, fapar_max)
Map2.addLayer(fm, {'min':0, 'max':1, 'palette':pal, 'opacity': 0.8}, 'fm')

## compute_gamma

This function computes the slope of the psychrometric constant from pressure given in Kpa. For this test, we need some pressure data. We can use the same ECMWF data:

In [21]:
sfp = 'surface_pressure' # in Pa (see https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY#bands)
surf = climate_img.select(sfp).divide(1000.0)  # convert from Pa to Kpa 
Map4 = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=9)
Map4.addLayer(surf, {'min':90, 'max':110, 'palette':pal, 'opacity': 0.8}, 'Surface pressure (KPa)')
Map4

Map(center=[30.236383, 38.530867], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [22]:
from geeet.ptjpl import compute_gamma
compute_gamma(105.15)

0.06989759826760286

In [23]:
Gamma = compute_gamma(surf, 'gamma')
Map4.addLayer(Gamma, {'min':0.06, 'max':0.075, 'palette':pal, 'opacity': 0.8}, 'Gamma')

## compute_delta

This function computes the slope of the relationship between saturation vapour pressure and air temperature.
See: http://www.fao.org/3/x0490e/x0490e07.htm#air%20temperature

To test this function we can again use the sample image `tair`

In [24]:
from geeet.ptjpl import compute_delta
compute_delta(18.8569)

0.1360154628370853

In [25]:
Delta = compute_delta(tair, 'Delta')
Map3.addLayer(Delta, {'min':0.0444, 'max':0.3107, 'palette':pal, 'opacity': 0.8}, 'Delta FAO')

## compute_apt_delta_gamma

This function computes the Priestley Taylor (PT) term `a*delta/(delta + gamma)`, where a is the Priestley-Taylor coefficient (a=1.26). Here we use the same temperature and pressure images as above. We will prepare a map with temperature and pressure for convenience of comparison.

In [26]:
Map5 = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=9)
Map5.addLayer(tair, {'min':0, 'max':35, 'palette':pal, 'opacity': 0.8}, 'Air temperature (2m)')
Map5.addLayer(surf, {'min':90, 'max':110, 'palette':pal, 'opacity': 0.8}, 'Surface pressure (KPa)')
Map5

Map(center=[30.236383, 38.530867], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [27]:
from geeet.ptjpl import compute_apt_delta_gamma
compute_apt_delta_gamma(14.728, 94.9633)

0.7955069676459955

In [28]:
APT = compute_apt_delta_gamma(tair, surf, 'APT')
Map5.addLayer(APT, {'min':0.6, 'max':0.9, 'palette':pal, 'opacity': 0.8}, 'APT term')

## compute_LAI

This function computes LAI from NDVI. We will use the same test NDVI data as above. Note: in the MEWA project, we will use our own LAI based on REGFLEC.

In [29]:
from geeet.ptjpl import compute_lai
compute_lai(0.79)

2.6941472959332184

In [30]:
LAI = compute_lai(NDVI, band_name = 'LAI')
Map2.addLayer(LAI, {'min':0, 'max':5, 'palette':ndvi_palette, 'opacity': 0.8}, 'LAI')

## compute_rnc and compute_rns

`compute_rns` computes the net radiation to the soil from net radiation and LAI. 

`compute_rnc` computes the net radiation to the canopy (residual from net radiation and rns)

In [31]:
from geeet.ptjpl import compute_rns, compute_rnc
compute_rns(264.487, 2.857)

47.63615828737228

In [32]:
compute_rnc(264.487,47.636)

216.85100000000003

For this test, we need some net radiation data. Let's use `surface_net_solar_radiation` from the hourly ECMWF data. 

In [33]:
net = 'surface_net_solar_radiation' # in J/m2 (see https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY#bands)
rnet = climate_img.select(net).divide(3600.0*13)  # convert from J/m2 to W/m2 (this variable is accumulated from the start of the simulation - 00:00 every day)
Map6 = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=9)
Map6.addLayer(rnet, {'min':0, 'max':500, 'palette':pal, 'opacity': 0.8}, 'Net solar radiation (W/m2)')
Map6

Map(center=[30.236383, 38.530867], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [34]:
RNS = compute_rns(rnet, LAI, 'rn soil')
RNC = compute_rnc(rnet, RNS, 'rn canopy')
Map6.addLayer(RNS, {'min':0, 'max':500, 'palette':pal, 'opacity': 0.8}, 'Rn soil (W/m2)')
Map6.addLayer(RNC, {'min':0, 'max':500, 'palette':pal, 'opacity': 0.8}, 'Rn canopy (W/m2)')
Map6.addLayer(LAI, {'min':0, 'max':5, 'palette':ndvi_palette, 'opacity': 0.8}, 'LAI')

## compute_vpd

Computes vapour pressure deficit from relative humidity and air temperature.

In [35]:
from geeet.ptjpl import compute_vpd
compute_vpd(10.9, 29.1088)

3.5915606007523957

In [36]:
RH = test_img.select(rh_band)
VPD = compute_vpd(RH, tair, 'VPD')
Map7 = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=7)
Map7.addLayer(RH, {'min':0, 'max':100, 'palette':["440154","3a528b","20908d","5dc962","fde725"], 'opacity': 0.8}, 'Relative humidity')
Map7.addLayer(tair, {'min':0, 'max':35, 'palette':pal, 'opacity': 0.8}, 'Air temperature (2m)')
Map7.addLayer(VPD,  {'min':0, 'max':5, 'palette':pal, 'opacity': 0.8}, 'Vapour pressure deficit (kPa)')
Map7

Map(center=[30.236383, 38.530867], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

## compute_fsm

Computes the soil moisture constrainf "fsm" from RH and air temperature.

In [37]:
from geeet.ptjpl import compute_fsm
compute_fsm(36, 17.7788)

array(0.26424006)

In [38]:
FSM = compute_fsm(RH, tair, band_name = 'Soil moisture constraint')
Map7.addLayer(FSM, {'min':0, 'max':1, 'opacity': 0.8}, 'Soil moisture constraint')

## compute_tnoon

Computes the solar noon. 
Here we need to create a longitude representation of earth. That is easy, but we also need the "standard meridian". It corresponds to every 15 degrees east of the prime meridian (I think?)..

Let's start by mapping longitude:

In [39]:
longitude = ee.Image.pixelLonLat().select('longitude')
#utmZones = longitude.add(180).divide(6).int()
stdMerid = longitude.add(187.5).divide(15).int().multiply(15).subtract(180)
Map8 = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=7)
Map8.addLayer(longitude, {'min':-180, 'max':180,'opacity':0.5}, 'longitude')
#Map8.addLayer(utmZones, {'min':0, 'max': 60}, 'UTM zones')
Map8.addLayer(stdMerid, {'min':-180, 'max': 180,'opacity':0.7}, 'Standard meridian?')
Map8

Map(center=[30.236383, 38.530867], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [40]:
date.format('yyyy DDD').getInfo()

'2017 105'

In [41]:
from geeet.ptjpl import compute_tnoon
compute_tnoon(39.99, 45, 105)

12.336617624901423

In [42]:
tnoon = compute_tnoon(longitude, stdMerid, 105, band_name = 'time of solar noon')
Map8.addLayer(tnoon, {'min':12,'max':13, 'opacity':0.7}, 'time of solar noon')

## compute_g

This function computes the soil heat flux based on Santanello and Friedl's parameterization.

In [43]:
from geeet.ptjpl import compute_g
compute_g(11, 12.3, 200)

53.815986651786844

In [44]:
G = compute_g(11, tnoon, RNS, band_name='soil heat flux')
Map6.addLayer(G, {'min':0,'max':500, 'palette': pal, 'opacity':0.7}, 'Soil heat flux')